### 1. Подготовка и загрузка данных

В этом ноутбуке мы объединим отфильтрованные датасеты всех супермаркетов и проведём финальную очистку данных.

In [2]:
import pandas as pd

Сначала загружаем списки продуктов для каждого супермаркета:

In [3]:
pyaterochka = pd.read_csv('filtered_products-2025-03-03-pyaterochka.csv')
lenta = pd.read_csv('filtered_products-2025-03-04-lenta.csv')
winmart = pd.read_csv('filtered_products-2025-03-06-winmart.csv')
coop = pd.read_csv('filtered_products-2025-03-07-coop.csv')

Каждая запись о товаре содержит как исходную, так и нормализованную цену, информацию о количестве (вес, объем или количество единиц), тип продукта (например, рис, молоко, чай) и название супермаркета.

### 2. Базовая очистка и подготовка данных

Мы удаляем индексные столбцы, оставшиеся после экспорта в CSV, и приводим названия столбцов к единому виду:

In [4]:
pyaterochka = pyaterochka.drop('id', axis=1).rename(columns={'pricing_unit': 'uom'})
lenta = lenta.drop(lenta.columns[0], axis=1).rename(columns={'pricing_unit': 'uom'})
winmart = winmart.drop(winmart.columns[0], axis=1)
coop = coop.drop(coop.columns[0], axis=1)

Данные всех товаров объединяем в один датафрейм.

In [5]:
all_products = pd.concat([pyaterochka, lenta, winmart, coop], ignore_index=True)

Быстрая проверка:

In [6]:
all_products

,name,price,uom,product_type,supermarket,weight,price_kg,number_of_units,price_unit,volume,price_lit
0,Багет традиционный 230г,45.99,230 г,bread,Pyaterochka,230.0,1.999565e+02,NaN,NaN,NaN,NaN
1,Багет Фитнес 200г,49.99,200 г,bread,Pyaterochka,200.0,2.499500e+02,NaN,NaN,NaN,NaN
2,Багет мини 120г,25.99,120 г,bread,Pyaterochka,120.0,2.165833e+02,NaN,NaN,NaN,NaN
3,Хлеб Маг с семечками 390г,75.99,390 г,bread,Pyaterochka,390.0,1.948462e+02,NaN,NaN,NaN,NaN
4,Огурцы среднеплодные,175.99,Цена за 1 кг,cucumber,Pyaterochka,1000.0,1.759900e+02,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2431,Nước mắm Nam Ngư 3in1 chai 10g/l – 750ml,52500.00,chai,fish_sauce,Co.op,10.0,5.250000e+06,NaN,NaN,750.0,70000.0
2432,Nước mắm cá cơm Hạnh Phúc 60 độ đạm 250ml,70200.00,chai,fish_sauce,Co.op,NaN,NaN,NaN,NaN,250.0,280800.0
2433,Gạo thơm ST25 plus lúa tôm Neptune 5kg,252000.00,bịch,rice,Co.op,5000.0,5.040000e+04,NaN,NaN,NaN,NaN
2434,Cam sành Coop Select túi kg,24900.00,kg,orange,Co.op,1000.0,2.490000e+04,NaN,NaN,NaN,NaN


### 3. Ручные исправления и корректировки

На основе ручной проверки удаляем ошибочные или неподходящие строки и добавляем недостающие товары.

In [ ]:
# удаляем нерелевантные товары, пропущенные на этапе фильтрации
all_products = all_products.drop([684, 224]) # бурый рис
all_products = all_products.drop(1877) # неподходящий тип яиц
all_products = all_products.drop(1683) # апельсины, у которых неизвестен вес
all_products = all_products.drop([1470, 1506]) # концентрированное молоко, а не сгущенное с сахаром
all_products = all_products.drop([108, 124, 129, 114, 118, 115, 131, 130, 421, 422, 419, 420, 418, 117, 1662]) # питьевой йогурт
all_products = all_products.drop([575, 566, 565, 564, 365, 360, 359, 560, 559, 576, 506]) # кофе в дрип-пакетах
all_products = all_products.drop([1294, 1297, 292, 289, 1304, 2310]) # не белый сахар или жидкий сахар (2310)
all_products = all_products.drop([443]) # непшеничные спагетти
all_products = all_products.drop([1499, 1495, 1493, 1498, 1489, 1497, 1478, 202]) # нерисовая лапша

# добавляем одну позицию баклажана для числа, не было в исходном наборе данных, но был в наличии в магазине
all_products.loc[10000] = ['Cà tím màng co', 25500, 'kg', 'eggplant', 'Co.op', 1000, 25500, None, None, None, None]

> **Примечание:** Указанные индексы соответствуют версии данных, обработанной в этом ноутбуке. Если в исходные данные будут внесены изменения, эти индексы могут больше не указывать на те же товары.

Затем мы исправляем проблемы с нормализацией цен и ошибочными значениями (например, отсутствующий вес, некорректный объём).

In [8]:
# вручную указываем количество для товаров, у которых не удалось нормализовать цену (все значения NaN)
all_products.loc[1660, 'weight'] = 100
all_products.loc[1736, 'weight'] = 250
all_products.loc[1800, 'volume'] = 500
all_products.loc[1819, 'weight'] = 500

# товары, которых нормализовались неверно (ошибочное количество)
all_products.loc[454, 'weight'] = 300
all_products.loc[1890, 'number_of_units'] = 10
all_products.loc[9, 'weight'] = 220
all_products.loc[1988, 'volume'] = 600 # на самом деле это упаковка 3x200ml
all_products.loc[1714, 'volume'] = 12000 # на самом деле это упаковка из 24 шт
all_products.loc[1789, 'volume'] = 600 # на самом деле это упаковка из 12 шт
all_products.loc[1792, 'volume'] = 750

# "весовым" товарам из Ленты при нормализации был назначен вес 1 кг, но фактический вес мог быть другим, он был указан в другом поле
# исправляем эти веса вручную
all_products.loc[598, 'weight'] = 300
all_products.loc[478, 'weight'] = 400
all_products.loc[[446, 453, 456, 459, 460, 461, 462, 463, 464, 467, 468, 469, 476, 488, 599, 728, 731, 733], 'weight'] = 500
all_products.loc[610, 'weight'] = 600
all_products.loc[734, 'weight'] = 700
all_products.loc[448, 'weight'] = 900
all_products.loc[736, 'weight'] = 950
all_products.loc[729, 'weight'] = 1200
all_products.loc[[735, 1162], 'weight'] = 1400
all_products.loc[[458, 1160], 'weight'] = 1500
all_products.loc[730, 'weight'] = 1700
all_products.loc[732, 'weight'] = 1900
all_products.loc[455, 'weight'] = 2700
all_products.loc[451, 'weight'] = 3000

# добавляем недостающие веса для некоторых чаёв
all_products.loc[[737, 739, 741, 742, 743, 744, 759, 760, 764, 771, 783, 784, 1097, 1003], 'weight'] = 200
all_products.loc[[745, 781, 1053, 1015, 866], 'weight'] = 45
all_products.loc[[746, 747, 752, 755, 756, 762, 763, 765, 767, 769, 779, 780, 782, 785, 1112, 1159, 961], 'weight'] = 50
all_products.loc[[748, 1089], 'weight'] = 180
all_products.loc[[753, 754, 772, 1118, 1149, 1038], 'weight'] = 37.5
all_products.loc[[770, 773, 1024], 'weight'] = 36
all_products.loc[[1471], 'weight'] = 100
all_products.loc[[1133, 986, 898], 'weight'] = 40
all_products.loc[[1123], 'weight'] = 24
all_products.loc[[1107], 'weight'] = 42.5
all_products.loc[[1075], 'weight'] = 27
all_products.loc[[1064], 'weight'] = 150
all_products.loc[[1058], 'weight'] = 34
all_products.loc[[1044], 'weight'] = 170
all_products.loc[[1020], 'weight'] = 170

### 4. Финальная нормализация цен

После того, как все ручные корректировки завершены, пересчитываем заново нормализованные цены.

In [9]:
all_products.loc[all_products['weight'].notna(), 'price_kg'] = all_products['price'] / all_products['weight'] * 1000
all_products.loc[all_products['number_of_units'].notna(), 'price_unit'] = all_products['price'] / all_products['number_of_units']
all_products.loc[all_products['volume'].notna(), 'price_lit'] = all_products['price'] / all_products['volume'] * 1000

Пара дополнительных корректировок для заполнения оставшихся пробелов.

- Для полноценного сравнения все молоко должно иметь объём в литрах, поэтому отсутствующие объёмы вычисляем с использованием средней плотности (1,03 г/мл).
- Чаи, у которых не указан вес непригодны для сравнения цен — некоторым присвоили вес вручну, остальные удаляем.

In [10]:
# конвертируем вес молока в объем (плотность ≈ 1.03 г/мл)
all_products.loc[(all_products['product_type'] == 'milk') & (all_products['volume'].isna()), 'volume'] = all_products['weight'] / 1.03
all_products.loc[all_products['volume'].notna(), 'price_lit'] = all_products['price'] / all_products['volume'] * 1000
# удаляем чаи без веса (для некоторых указали вес вручную ранее)
all_products = all_products.loc[~((all_products['product_type'].isin(['black_tea', 'green_tea'])) & (all_products['weight'].isna()))]

### 5. Финальная очистка и форматирование

Для каждого типа продукта будет использоваться единственный ценовой показатель: например, для риса и овощей — price_kg, для воды — price_lit, для яиц — price_unit. Чтобы избежать путаницы в дальнейшем анализе, удалим лишние значения в столбцах, связанных с количеством (например, удаляем вес и price_kg для жидкостей или number_of_units для гречки).

Также создаём новый DataFrame для финальной очищенной версии данных.

In [12]:
clean_products = all_products

# сопоставляем типы продуктов с их ценовым показателем 
unit_types = ['egg'] # штучные товары
volume_types = ['water', 'sunflower_oil', 'soybean_oil', 'milk', 'fish_sauce'] # объемные товары
weight_items = ~clean_products['product_type'].isin(unit_types + volume_types) # весовые товары

# устанавливаем None в нерелевантных полях в зависимости от типа продукта
clean_products.loc[clean_products['product_type'].isin(unit_types), ['weight', 'price_kg', 'volume', 'price_lit']] = None
clean_products.loc[clean_products['product_type'].isin(volume_types), ['weight', 'price_kg', 'number_of_units', 'price_unit']] = None
clean_products.loc[weight_items, ['volume', 'price_lit', 'number_of_units', 'price_unit']] = None

Быстрая проверка:

In [22]:
clean_products.sample(10)

,name,price,uom,product_type,supermarket,weight,price_kg,number_of_units,price_unit,volume,price_lit
1667,Bắp cải trắng,15900.00,KG,cabbage,Winmart,1000.0,15900.000000,NaN,NaN,NaN,NaN
2247,Gạo thơm ST 25 thượng hạng Đồng Việt 5kg,194500.00,bịch,rice,Co.op,5000.0,38900.000000,NaN,NaN,NaN,NaN
1804,Nước mắm Cá Cơm Thuận Phát 60N chai 490ml,145000.00,CHA,fish_sauce,Winmart,NaN,NaN,NaN,NaN,490.0,295918.367347
1673,Hành tây,26900.00,KG,onion,Winmart,1000.0,26900.000000,NaN,NaN,NaN,NaN
258,Чай черный Лисма Насыщенный индийский 25х1.8г,55.99,25 шт,black_tea,Pyaterochka,45.0,1244.222222,NaN,NaN,NaN,NaN
1260,Вода питьевая ЧЕРНОГОЛОВКА артезианская негази...,97.89,2.5L,water,Lenta,NaN,NaN,NaN,NaN,2500.0,39.156000
1115,Чай зеленый ЧЕРНЫЙ ДРАКОН Китайский с имбирем ...,216.49,100г,green_tea,Lenta,100.0,2164.900000,NaN,NaN,NaN,NaN
231,Гречка Мистраль Фермерская 900г,169.99,900 г,buckwheat,Pyaterochka,900.0,188.877778,NaN,NaN,NaN,NaN
798,Чай черный HYLEYS Английский Аристократический...,740.24,500г,black_tea,Lenta,500.0,1480.480000,NaN,NaN,NaN,NaN
225,Рис Агро-альянс Экстра Элитный длиннозерный 900г,169.99,900 г,rice,Pyaterochka,900.0,188.877778,NaN,NaN,NaN,NaN


Наконец, сохраняем итоговый набор данных в новый файл.

In [11]:
clean_products.to_csv(f'clean_products-2025-03-12-local.csv')